<img src="https://github.com/IKNL/guidelines/blob/master/resources/logos/vantage6.png?raw=true" width=200 align="right">

# Hello vantage6 
_feature testing a node instance_

In [1]:
import rich

from os import getenv
from dotenv import load_dotenv
from rich.table import Table
from rich.text import Text
from rich.console import Console

from vantage6.client import Client

# loading environment variables in '.env' file. 
load_dotenv()
user = getenv('USER')
password = getenv('PASSWORD')
url = getenv('SERVER')
port = getenv('PORT')
path = getenv('API_PATH')

# Authenticate & Setup

In [2]:
client = Client(url, port, path)
client.authenticate(user, password)
client.setup_encryption(None)

# Collecting information
Before we can run any algorithm we need to collect some info about the collaboration, organization and nodes.

In [3]:
collaborations = client.collaboration.list(scope='global', fields=('id','name', 'encrypted'), include_metadata=False)

In [4]:
table = Table(title='Collaborations')
table.add_column("Id", justify="right", style="cyan", no_wrap=True)
table.add_column("Name", style="magenta")
table.add_column("Encrypted", justify="right", style="green")
for collaboration in collaborations:
    table.add_row(*list(map(str, collaboration.values())))
    
table

             Collaborations              
┏━━━━┳━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┓
┃ Id ┃ Name                 ┃ Encrypted ┃
┡━━━━╇━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━┩
│  1 │ testing              │     False │
│  2 │ tno-demo             │     False │
│  3 │ palliative_care_lumc │      True │
│  4 │ Lancelot             │     False │
└────┴──────────────────────┴───────────┘

In [5]:
# set this to the collaboration ID you want to use
COLLABORATION_ID = 1

In [6]:
collaboration = client.collaboration.get(COLLABORATION_ID, fields=('organizations','nodes'))

In [7]:
fields=('id', 'name')
organization_ids = [n.get('id') for n in collaboration['organizations']]
organizations = [client.organization.get(id_, fields=fields) for id_ in organization_ids]
table = Table(title='Organizations')
[table.add_column(column) for column in fields]
[table.add_row(*map(str, list(organization.values()))) for organization in organizations]
table

Organizations
┏━━━━┳━━━━━━┓
┃ id ┃ name ┃
┡━━━━╇━━━━━━┩
│ 1  │ root │
│ 2  │ IKNL │
└────┴──────┘

In [8]:
fields=('id', 'name', 'status', 'last_seen')
node_ids = [n.get('id') for n in collaboration['nodes']]
nodes = [client.node.get(id_, fields=fields) for id_ in node_ids]
table = Table(title='Nodes')
[table.add_column(column) for column in fields]
[table.add_row(*map(str, list(node.values()))) for node in nodes]
table

                                 Nodes                                  
┏━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ id ┃ name                ┃ status ┃ last_seen                        ┃
┡━━━━╇━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ 4  │ IKNL - testing Node │ online │ 2022-05-09T09:54:55.531484+00:00 │
│ 3  │ root - testing Node │ online │ 2022-05-09T09:54:56.656157+00:00 │
└────┴─────────────────────┴────────┴──────────────────────────────────┘

# Reset API keys
_Only do this if you do no longer have the API keys of the nodes_

In [9]:
#node = client.request(f"/recover/node", method="post", json={'id': 4})
#node

# Run the feature tester

In [10]:
from socketio import ClientNamespace, Client as SocketIO

In [11]:
client.request(f'result/603')['task']

{'id': 293, 'link': '/task/293', 'methods': ['DELETE', 'GET']}

In [12]:
console = Console()

class TasksNamespace(ClientNamespace):

    def on_status_update(self, result_id):

        # print("socket call!")
        result_id = result_id.get('result_id')
        # print(f'result/{result_id}')
        task_id = client.request(f'result/{result_id}').get('task').get('id')
        # print(task_id)
        task = client.task.get(task_id)
        # print(task)
        if task.get('complete') == True: 
            results = client.task.get(task.get("id"), include_results=True)['results']
            # print(results)
            # print the results per node
            for result in results:
                console.print('*'*80)
                console.print(result['log']) 

In [13]:
socket = SocketIO()
socket.register_namespace(TasksNamespace('/tasks'))

In [14]:
socket.connect(f'{url}:{port}', headers=client.headers)

In [15]:
socket.emit("join_room", f"collaboration_{COLLABORATION_ID}", namespace='/tasks')

In [18]:
# The feature tester image to be used
IMAGE = 'harbor2.vantage6.ai/infrastructure/hello-vantage6'
task = client.task.create(collaboration=COLLABORATION_ID, organizations=organization_ids, name='feature testing', image=IMAGE, description='Checking the node installation by sending a feature tester', input={})

********************************************************************************

--> Reading the environment variables
ENVIRONMENT: environ({'INPUT_FILE': '/mnt/data/task-000000617/input', 'HOSTNAME': 
'7e76b0f40a30', 'PYTHON_PIP_VERSION': '20.3.3', 'TOKEN_FILE': 
'/mnt/data/task-000000617/token', 'PORT': '80', 'HOME': '/root', 'GPG_KEY': 
'0D96DF4D4110E5C43FBFB17F2D347EA6AA65421D', 'TEMPORARY_FOLDER': '/mnt/tmp', 'API_PATH': '', 
'PYTHON_GET_PIP_URL': 
'https://github.com/pypa/get-pip/raw/5f38681f7f5872e4032860b54e9cc11cf0374932/get-pip.py', 
'PKG_NAME': 'v6-feature-tester-py', 'PATH': 
'/usr/local/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin', 'LANG': 
'C.UTF-8', 'DEFAULT_DATABASE_URI': '/mnt/data/default.csv', 'HOST': 'http://proxyserver', 
'PYTHON_VERSION': '3.7.9', 'PWD': '/', 'PYTHON_GET_PIP_SHA256': 
'6a0b13826862f33c13b614a921d36253bfa1ae779c5fbf569876f3585057e9d2', 'OUTPUT_FILE': 
'/mnt/data/task-000000617/output', 'DATABASE_URI': '/mnt/data/default.csv'})
--> Reading input file
INPUT FILE: b'\x80\x04}\x94.'
--> Writing to output file (contents: test)
--> Reading output file back and check
test
--> Reading token file
TOKEN: eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJpYXQiOjE2NTIwOTAxOTEsIm5iZiI6MTY1MjA5MDE5MSwia
nRpIjoiOThlOThiNWEtZjJhNy00MDQwLWI2NmUtMTkxYTgwYjE2MmVmIiwiaWRlbnRpdHkiOnsidHlwZSI6ImNvbnRhaW
5lciIsIm5vZGVfaWQiOjQsIm9yZ2FuaXphdGlvbl9pZCI6MiwiY29sbGFib3JhdGlvbl9pZCI6MSwidGFza19pZCI6MzA
wLCJpbWFnZSI6ImhhcmJvcjIudmFudGFnZTYuYWkvaW5mcmFzdHJ1Y3R1cmUvaGVsbG8tdmFudGFnZTYifSwiZnJlc2gi
OmZhbHNlLCJ0eXBlIjoiYWNjZXNzIiwidXNlcl9jbGFpbXMiOnsidHlwZSI6ImNvbnRhaW5lciIsInJvbGVzIjpbXX19.
G6vEMNLqXHOVt-EW7-zsScoLapNYlLC0mFyUiMUOX3Q
--> Test temporary volume
--> Writing to temporary file: /mnt/tmp/test.txt
--> Test that the temporary file is created
FILE CREATED: True
--> Test that we can reach the local proxy (and thereby the server)
--> Verify that the container has no internet connection
--> Connection error catched
--> Check that two ports have been published
debug
{'assigned_at': '2022-05-09T09:56:29.022377+00:00', 'finished_at': None, 'id': 617, 'input': 
'gAR9lC4=', 'log': None, 'node': {'id': 4, 'ip': '10.76.0.48', 'name': 'IKNL - testing Node',
'status': 'online'}, 'organization': 2, 'ports': [{'id': 219, 'label': 'port5', 'port': 
49178}, {'id': 221, 'label': 'port8', 'port': 49179}], 'result': None, 'started_at': 
'2022-05-09T09:56:29.921688+00:00', 'task': 300}
--> Found 2 port(s)
--> found 'port5':49178
--> found 'port8':49179
--> Check attached databases exists
--> Found database-path: DEFAULT_DATABASE_URI
--> database 'DEFAULT_DATABASE_URI' is reachable: /mnt/data/default.csv
{'environment': environ({'INPUT_FILE': '/mnt/data/task-000000617/input', 'HOSTNAME': 
'7e76b0f40a30', 'PYTHON_PIP_VERSION': '20.3.3', 'TOKEN_FILE': 
'/mnt/data/task-000000617/token', 'PORT': '80', 'HOME': '/root', 'GPG_KEY': 
'0D96DF4D4110E5C43FBFB17F2D347EA6AA65421D', 'TEMPORARY_FOLDER': '/mnt/tmp', 'API_PATH': '', 
'PYTHON_GET_PIP_URL': 
'https://github.com/pypa/get-pip/raw/5f38681f7f5872e4032860b54e9cc11cf0374932/get-pip.py', 
'PKG_NAME': 'v6-feature-tester-py', 'PATH': 
'/usr/local/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin', 'LANG': 
'C.UTF-8', 'DEFAULT_DATABASE_URI': '/mnt/data/default.csv', 'HOST': 'http://proxyserver', 
'PYTHON_VERSION': '3.7.9', 'PWD': '/', 'PYTHON_GET_PIP_SHA256': 
'6a0b13826862f33c13b614a921d36253bfa1ae779c5fbf569876f3585057e9d2', 'OUTPUT_FILE': 
'/mnt/data/task-000000617/output', 'DATABASE_URI': '/mnt/data/default.csv'}), 
'READ_INPUT_FILE': {'Success': True}, 'WRITE_READ_OUTPUT_FILE': {'Success': True}, 
'READ_TOKEN_FILE': {'Success': True}, 'TEMPORARY_VOLUME': {'Success': True}, 
'TEMPORARY_VOLUME_FILE_EXISTS': {'Success': True}, 'LOCAL_PROXY_CENTRAL_SERVER': {'Success': 
True}, 'ISOLATION_TEST': {'Success': True}, 'EXTERNAL_PORT_TEST': {'Success': True}}

********************************************************************************

--> Reading the environment variables
ENVIRONMENT: environ({'INPUT_FILE': '/mnt/data/task-000000616/input', 'HOSTNAME': 
'b5eb99717069', 'PYTHON_PIP_VERSION': '20.3.3', 'TOKEN_FILE': 
'/mnt/data/task-000000616/token', 'PORT': '80', 'HOME': '/root', 'GPG_KEY': 
'0D96DF4D4110E5C43FBFB17F2D347EA6AA65421D', 'TEMPORARY_FOLDER': '/mnt/tmp', 'API_PATH': '', 
'PYTHON_GET_PIP_URL': 
'https://github.com/pypa/get-pip/raw/5f38681f7f5872e4032860b54e9cc11cf0374932/get-pip.py', 
'PKG_NAME': 'v6-feature-tester-py', 'PATH': 
'/usr/local/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin', 'LANG': 
'C.UTF-8', 'DEFAULT_DATABASE_URI': '/mnt/data/default.csv', 'HOST': 'http://proxyserver', 
'PYTHON_VERSION': '3.7.9', 'PWD': '/', 'PYTHON_GET_PIP_SHA256': 
'6a0b13826862f33c13b614a921d36253bfa1ae779c5fbf569876f3585057e9d2', 'OUTPUT_FILE': 
'/mnt/data/task-000000616/output', 'DATABASE_URI': '/mnt/data/default.csv'})
--> Reading input file
INPUT FILE: b'\x80\x04}\x94.'
--> Writing to output file (contents: test)
--> Reading output file back and check
test
--> Reading token file
TOKEN: eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJpYXQiOjE2NTIwOTAxOTIsIm5iZiI6MTY1MjA5MDE5Miwia
nRpIjoiNjBlZDlhYTktYWZiOS00MzZlLTgyOGItZTA5NjliZTNiNDY4IiwiaWRlbnRpdHkiOnsidHlwZSI6ImNvbnRhaW
5lciIsIm5vZGVfaWQiOjMsIm9yZ2FuaXphdGlvbl9pZCI6MSwiY29sbGFib3JhdGlvbl9pZCI6MSwidGFza19pZCI6MzA
wLCJpbWFnZSI6ImhhcmJvcjIudmFudGFnZTYuYWkvaW5mcmFzdHJ1Y3R1cmUvaGVsbG8tdmFudGFnZTYifSwiZnJlc2gi
OmZhbHNlLCJ0eXBlIjoiYWNjZXNzIiwidXNlcl9jbGFpbXMiOnsidHlwZSI6ImNvbnRhaW5lciIsInJvbGVzIjpbXX19.
xuciHC2LYbrq-l4MGmzZrdudQQ1nSmrzWx5b5cCikoE
--> Test temporary volume
--> Writing to temporary file: /mnt/tmp/test.txt
--> Test that the temporary file is created
FILE CREATED: True
--> Test that we can reach the local proxy (and thereby the server)
--> Verify that the container has no internet connection
--> Connection error catched
--> Check that two ports have been published
debug
{'assigned_at': '2022-05-09T09:56:29.022377+00:00', 'finished_at': None, 'id': 617, 'input': 
'gAR9lC4=', 'log': None, 'node': {'id': 4, 'ip': '10.76.0.48', 'name': 'IKNL - testing Node',
'status': 'online'}, 'organization': 2, 'ports': [{'id': 219, 'label': 'port5', 'port': 
49178}, {'id': 221, 'label': 'port8', 'port': 49179}], 'result': None, 'started_at': 
'2022-05-09T09:56:29.921688+00:00', 'task': 300}
--> Found 2 port(s)
--> found 'port5':49178
--> found 'port8':49179
--> Check attached databases exists
--> Found database-path: DEFAULT_DATABASE_URI
--> database 'DEFAULT_DATABASE_URI' is reachable: /mnt/data/default.csv
{'environment': environ({'INPUT_FILE': '/mnt/data/task-000000616/input', 'HOSTNAME': 
'b5eb99717069', 'PYTHON_PIP_VERSION': '20.3.3', 'TOKEN_FILE': 
'/mnt/data/task-000000616/token', 'PORT': '80', 'HOME': '/root', 'GPG_KEY': 
'0D96DF4D4110E5C43FBFB17F2D347EA6AA65421D', 'TEMPORARY_FOLDER': '/mnt/tmp', 'API_PATH': '', 
'PYTHON_GET_PIP_URL': 
'https://github.com/pypa/get-pip/raw/5f38681f7f5872e4032860b54e9cc11cf0374932/get-pip.py', 
'PKG_NAME': 'v6-feature-tester-py', 'PATH': 
'/usr/local/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin', 'LANG': 
'C.UTF-8', 'DEFAULT_DATABASE_URI': '/mnt/data/default.csv', 'HOST': 'http://proxyserver', 
'PYTHON_VERSION': '3.7.9', 'PWD': '/', 'PYTHON_GET_PIP_SHA256': 
'6a0b13826862f33c13b614a921d36253bfa1ae779c5fbf569876f3585057e9d2', 'OUTPUT_FILE': 
'/mnt/data/task-000000616/output', 'DATABASE_URI': '/mnt/data/default.csv'}), 
'READ_INPUT_FILE': {'Success': True}, 'WRITE_READ_OUTPUT_FILE': {'Success': True}, 
'READ_TOKEN_FILE': {'Success': True}, 'TEMPORARY_VOLUME': {'Success': True}, 
'TEMPORARY_VOLUME_FILE_EXISTS': {'Success': True}, 'LOCAL_PROXY_CENTRAL_SERVER': {'Success': 
True}, 'ISOLATION_TEST': {'Success': True}, 'EXTERNAL_PORT_TEST': {'Success': True}}

********************************************************************************

--> Reading the environment variables
ENVIRONMENT: environ({'INPUT_FILE': '/mnt/data/task-000000617/input', 'HOSTNAME': 
'7e76b0f40a30', 'PYTHON_PIP_VERSION': '20.3.3', 'TOKEN_FILE': 
'/mnt/data/task-000000617/token', 'PORT': '80', 'HOME': '/root', 'GPG_KEY': 
'0D96DF4D4110E5C43FBFB17F2D347EA6AA65421D', 'TEMPORARY_FOLDER': '/mnt/tmp', 'API_PATH': '', 
'PYTHON_GET_PIP_URL': 
'https://github.com/pypa/get-pip/raw/5f38681f7f5872e4032860b54e9cc11cf0374932/get-pip.py', 
'PKG_NAME': 'v6-feature-tester-py', 'PATH': 
'/usr/local/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin', 'LANG': 
'C.UTF-8', 'DEFAULT_DATABASE_URI': '/mnt/data/default.csv', 'HOST': 'http://proxyserver', 
'PYTHON_VERSION': '3.7.9', 'PWD': '/', 'PYTHON_GET_PIP_SHA256': 
'6a0b13826862f33c13b614a921d36253bfa1ae779c5fbf569876f3585057e9d2', 'OUTPUT_FILE': 
'/mnt/data/task-000000617/output', 'DATABASE_URI': '/mnt/data/default.csv'})
--> Reading input file
INPUT FILE: b'\x80\x04}\x94.'
--> Writing to output file (contents: test)
--> Reading output file back and check
test
--> Reading token file
TOKEN: eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJpYXQiOjE2NTIwOTAxOTEsIm5iZiI6MTY1MjA5MDE5MSwia
nRpIjoiOThlOThiNWEtZjJhNy00MDQwLWI2NmUtMTkxYTgwYjE2MmVmIiwiaWRlbnRpdHkiOnsidHlwZSI6ImNvbnRhaW
5lciIsIm5vZGVfaWQiOjQsIm9yZ2FuaXphdGlvbl9pZCI6MiwiY29sbGFib3JhdGlvbl9pZCI6MSwidGFza19pZCI6MzA
wLCJpbWFnZSI6ImhhcmJvcjIudmFudGFnZTYuYWkvaW5mcmFzdHJ1Y3R1cmUvaGVsbG8tdmFudGFnZTYifSwiZnJlc2gi
OmZhbHNlLCJ0eXBlIjoiYWNjZXNzIiwidXNlcl9jbGFpbXMiOnsidHlwZSI6ImNvbnRhaW5lciIsInJvbGVzIjpbXX19.
G6vEMNLqXHOVt-EW7-zsScoLapNYlLC0mFyUiMUOX3Q
--> Test temporary volume
--> Writing to temporary file: /mnt/tmp/test.txt
--> Test that the temporary file is created
FILE CREATED: True
--> Test that we can reach the local proxy (and thereby the server)
--> Verify that the container has no internet connection
--> Connection error catched
--> Check that two ports have been published
debug
{'assigned_at': '2022-05-09T09:56:29.022377+00:00', 'finished_at': None, 'id': 617, 'input': 
'gAR9lC4=', 'log': None, 'node': {'id': 4, 'ip': '10.76.0.48', 'name': 'IKNL - testing Node',
'status': 'online'}, 'organization': 2, 'ports': [{'id': 219, 'label': 'port5', 'port': 
49178}, {'id': 221, 'label': 'port8', 'port': 49179}], 'result': None, 'started_at': 
'2022-05-09T09:56:29.921688+00:00', 'task': 300}
--> Found 2 port(s)
--> found 'port5':49178
--> found 'port8':49179
--> Check attached databases exists
--> Found database-path: DEFAULT_DATABASE_URI
--> database 'DEFAULT_DATABASE_URI' is reachable: /mnt/data/default.csv
{'environment': environ({'INPUT_FILE': '/mnt/data/task-000000617/input', 'HOSTNAME': 
'7e76b0f40a30', 'PYTHON_PIP_VERSION': '20.3.3', 'TOKEN_FILE': 
'/mnt/data/task-000000617/token', 'PORT': '80', 'HOME': '/root', 'GPG_KEY': 
'0D96DF4D4110E5C43FBFB17F2D347EA6AA65421D', 'TEMPORARY_FOLDER': '/mnt/tmp', 'API_PATH': '', 
'PYTHON_GET_PIP_URL': 
'https://github.com/pypa/get-pip/raw/5f38681f7f5872e4032860b54e9cc11cf0374932/get-pip.py', 
'PKG_NAME': 'v6-feature-tester-py', 'PATH': 
'/usr/local/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin', 'LANG': 
'C.UTF-8', 'DEFAULT_DATABASE_URI': '/mnt/data/default.csv', 'HOST': 'http://proxyserver', 
'PYTHON_VERSION': '3.7.9', 'PWD': '/', 'PYTHON_GET_PIP_SHA256': 
'6a0b13826862f33c13b614a921d36253bfa1ae779c5fbf569876f3585057e9d2', 'OUTPUT_FILE': 
'/mnt/data/task-000000617/output', 'DATABASE_URI': '/mnt/data/default.csv'}), 
'READ_INPUT_FILE': {'Success': True}, 'WRITE_READ_OUTPUT_FILE': {'Success': True}, 
'READ_TOKEN_FILE': {'Success': True}, 'TEMPORARY_VOLUME': {'Success': True}, 
'TEMPORARY_VOLUME_FILE_EXISTS': {'Success': True}, 'LOCAL_PROXY_CENTRAL_SERVER': {'Success': 
True}, 'ISOLATION_TEST': {'Success': True}, 'EXTERNAL_PORT_TEST': {'Success': True}}

********************************************************************************

--> Reading the environment variables
ENVIRONMENT: environ({'INPUT_FILE': '/mnt/data/task-000000616/input', 'HOSTNAME': 
'b5eb99717069', 'PYTHON_PIP_VERSION': '20.3.3', 'TOKEN_FILE': 
'/mnt/data/task-000000616/token', 'PORT': '80', 'HOME': '/root', 'GPG_KEY': 
'0D96DF4D4110E5C43FBFB17F2D347EA6AA65421D', 'TEMPORARY_FOLDER': '/mnt/tmp', 'API_PATH': '', 
'PYTHON_GET_PIP_URL': 
'https://github.com/pypa/get-pip/raw/5f38681f7f5872e4032860b54e9cc11cf0374932/get-pip.py', 
'PKG_NAME': 'v6-feature-tester-py', 'PATH': 
'/usr/local/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin', 'LANG': 
'C.UTF-8', 'DEFAULT_DATABASE_URI': '/mnt/data/default.csv', 'HOST': 'http://proxyserver', 
'PYTHON_VERSION': '3.7.9', 'PWD': '/', 'PYTHON_GET_PIP_SHA256': 
'6a0b13826862f33c13b614a921d36253bfa1ae779c5fbf569876f3585057e9d2', 'OUTPUT_FILE': 
'/mnt/data/task-000000616/output', 'DATABASE_URI': '/mnt/data/default.csv'})
--> Reading input file
INPUT FILE: b'\x80\x04}\x94.'
--> Writing to output file (contents: test)
--> Reading output file back and check
test
--> Reading token file
TOKEN: eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJpYXQiOjE2NTIwOTAxOTIsIm5iZiI6MTY1MjA5MDE5Miwia
nRpIjoiNjBlZDlhYTktYWZiOS00MzZlLTgyOGItZTA5NjliZTNiNDY4IiwiaWRlbnRpdHkiOnsidHlwZSI6ImNvbnRhaW
5lciIsIm5vZGVfaWQiOjMsIm9yZ2FuaXphdGlvbl9pZCI6MSwiY29sbGFib3JhdGlvbl9pZCI6MSwidGFza19pZCI6MzA
wLCJpbWFnZSI6ImhhcmJvcjIudmFudGFnZTYuYWkvaW5mcmFzdHJ1Y3R1cmUvaGVsbG8tdmFudGFnZTYifSwiZnJlc2gi
OmZhbHNlLCJ0eXBlIjoiYWNjZXNzIiwidXNlcl9jbGFpbXMiOnsidHlwZSI6ImNvbnRhaW5lciIsInJvbGVzIjpbXX19.
xuciHC2LYbrq-l4MGmzZrdudQQ1nSmrzWx5b5cCikoE
--> Test temporary volume
--> Writing to temporary file: /mnt/tmp/test.txt
--> Test that the temporary file is created
FILE CREATED: True
--> Test that we can reach the local proxy (and thereby the server)
--> Verify that the container has no internet connection
--> Connection error catched
--> Check that two ports have been published
debug
{'assigned_at': '2022-05-09T09:56:29.022377+00:00', 'finished_at': None, 'id': 617, 'input': 
'gAR9lC4=', 'log': None, 'node': {'id': 4, 'ip': '10.76.0.48', 'name': 'IKNL - testing Node',
'status': 'online'}, 'organization': 2, 'ports': [{'id': 219, 'label': 'port5', 'port': 
49178}, {'id': 221, 'label': 'port8', 'port': 49179}], 'result': None, 'started_at': 
'2022-05-09T09:56:29.921688+00:00', 'task': 300}
--> Found 2 port(s)
--> found 'port5':49178
--> found 'port8':49179
--> Check attached databases exists
--> Found database-path: DEFAULT_DATABASE_URI
--> database 'DEFAULT_DATABASE_URI' is reachable: /mnt/data/default.csv
{'environment': environ({'INPUT_FILE': '/mnt/data/task-000000616/input', 'HOSTNAME': 
'b5eb99717069', 'PYTHON_PIP_VERSION': '20.3.3', 'TOKEN_FILE': 
'/mnt/data/task-000000616/token', 'PORT': '80', 'HOME': '/root', 'GPG_KEY': 
'0D96DF4D4110E5C43FBFB17F2D347EA6AA65421D', 'TEMPORARY_FOLDER': '/mnt/tmp', 'API_PATH': '', 
'PYTHON_GET_PIP_URL': 
'https://github.com/pypa/get-pip/raw/5f38681f7f5872e4032860b54e9cc11cf0374932/get-pip.py', 
'PKG_NAME': 'v6-feature-tester-py', 'PATH': 
'/usr/local/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin', 'LANG': 
'C.UTF-8', 'DEFAULT_DATABASE_URI': '/mnt/data/default.csv', 'HOST': 'http://proxyserver', 
'PYTHON_VERSION': '3.7.9', 'PWD': '/', 'PYTHON_GET_PIP_SHA256': 
'6a0b13826862f33c13b614a921d36253bfa1ae779c5fbf569876f3585057e9d2', 'OUTPUT_FILE': 
'/mnt/data/task-000000616/output', 'DATABASE_URI': '/mnt/data/default.csv'}), 
'READ_INPUT_FILE': {'Success': True}, 'WRITE_READ_OUTPUT_FILE': {'Success': True}, 
'READ_TOKEN_FILE': {'Success': True}, 'TEMPORARY_VOLUME': {'Success': True}, 
'TEMPORARY_VOLUME_FILE_EXISTS': {'Success': True}, 'LOCAL_PROXY_CENTRAL_SERVER': {'Success': 
True}, 'ISOLATION_TEST': {'Success': True}, 'EXTERNAL_PORT_TEST': {'Success': True}}